## Cascadia RSV A/B swab dates and serum collection date plots
**The goal of this script is to create a single plot for each patient that contains:**
- a timeline of the dates that they tested POSITIVE for RSV A or RSV B (see see computational_notebooks/gjuviler/rsv_imprinting/01-data/Imprinting_Sera2 - column name: '0a_rsv_b' or '0a_rsv_a'. A 1 denotes a positive swab.)
- a timeline of the dates that serum was collected that WE HAVE RIGHT NOW (see computational_notebooks/gjuviler/rsv_imprinting/Bloom_Simonich_CASCADIA_Oct2025_Samples.xlsx)
- the date and outcome of pre-F binding antibody tests (see computational_notebooks/gjuviler/rsv_imprinting/Imprinting_Sera2 - column name: 'ar_rsv_pre_f')
    - not sure if this should be a number or convert the number to a simple yes/no if binding occurred or not
    - currently on Teagan's plots, this is on a timeline of patient visit (redcap repeat instance), but we would rather have a date
- the date and outcome of neutralization titer assays (see computational_notebooks/gjuviler/rsv_imprinting/Imprinting_Sera2 - column name: 'ar_rsva_nd50')
    - again, this is currently on a patient visit timeline, but we want the date
    - there is only rsva neut data

**Previous work**
- Teagan has two notebooks in her comp notebook (see computational_notebooks/tmcmahon/2025/RSV_imprinting/02_notebooks) that create plots
- These notebooks are extremely long and it's hard to tell exactly what is going on. The outputs are located in the output folder, and are fairly useful. However, the changes mentioned above need to be made
- There is a Data dictionary for the swab data in the sample_info folder in 01-data in Teagan's comp notebook. 
- There is a data dictionary for the serum data in 01-data/sample_info/CASCADIA_DataMart_dd_shared_20240830 folder in Teagan's comp. notebook. 


# Next steps
The next step is getting the binding and titer data plotted by date. According to their email, ar_sars_msd_sc_date is the collection date and ar_date is the date it was run. There is also ar_sars_msd_sc_date_mmwr and ar_sars_msd_sc_date_epi (which appers to be in year - week format). In the ar_sars_msd_sc_date column, which should be the collection date, the numbers are unclear. For example, for ptid 20000401, the dates are 23332 and 22912. I have no idea what dates these are. I might need to dig around Teagan's notebook to see if there is an original file, and maybe the dates got messed up by excel somehow?

I will also need to double check which rsv column I should be using to determine the rsv positive swabs. 

### serum dates: 
- according to the data dictionary (tmcmahon2025/RSV_imprinting/01-data/sample_info/CASCADIA_DataMart_dd_shared_20240830), the date format is in SAS format, which is the number of days between January 1, 1960, and the specified date. If that is true:
- 23332 = 18 Nov. 2023, corresponds to 2023 week 46 which makes sense
- 22909 = 21 Sep. 2022, corresponds to 2022 week 38 which also makes sense
This SAS format appears to be correct - next step is to create a script that converts it to a readable date format (YYYY-MM-DD)

### Import necessary components

In [ ]:
import math
import os
import altair as alt
import numpy as np
import pandas as pd
from pathlib import Path

#os.chdir('..')
print(os.getcwd())


/fh/fast/bloom_j/computational_notebooks/gjuviler/rsv_imprinting


### Read the current data

In [57]:


selected_child = pd.read_csv('01-data/Bloom_Simonich_CASCADIA_Oct2025_Samples_children.csv')
selected_child['ptid'] = pd.to_numeric(selected_child['ptid'], downcast='integer', errors='coerce')     #convert the ptid column to int

selected_adult = pd.read_csv('01-data/Bloom_Simonich_CASCADIA_Oct2025_Samples_adult.csv')

sera = pd.read_csv('01-data/Imprinting_Sera2.csv')       #sera data
swab = pd.read_csv('01-data/Imprinting_swab2.csv')       #swab data
sera_swab = '01-data/Imprinting_sera_swab'      #this is the new dataframe into which I will import the necessary information

### input names for saved chart/timelines

In [58]:
date = '26.02.25'

sera_filtered_converted_date = '01-data/Imprinting_Sera2_converted_dates.csv'       #save the filtered sera dataframe with dates converted to YYYY-MM-DD from SAS
aliquot_swab_timeline = '03-results/26.02.25.selected_aliquot_swab_timelines.html'    #save the swab+aliquot timelines as an html
timeline_stacked = '03-results/26.02.26.stacked_timeline.html'

### First, let's create a new dataframe of serum that contains only the patient ids that we have on hand, then filters for ptid, aliquot id, rsva/b outcome, ar_sars_msd_sc_date, ar_rsv_pre_f, ar_rsva_nd50

In [59]:
df_aliquots_child = selected_child[['household_id', 'ptid', 'aliquot_id', 'collect_dt']]    #creates a new df with just these 4 columns

#dataframe of our selected ptids + dates
selected_ptids = []        #creates a list of patient ids (in int format) that we have that we can use to select the correct sera data
for index, ptid in enumerate(df_aliquots_child.iloc[:, 1]):
    if ptid != 'EMPTY':     #some of the rows have EMPTY where the ptid should be 
        selected_ptids.append(ptid)

sera_filtered_ptid = sera[sera['ptid'].isin(selected_ptids)]
sera_filtered = sera_filtered_ptid[['ptid', 'aliquot_id', 'rsv_a_outcome', 'rsv_b_outcome', 'ar_sars_msd_sc_date', 'ar_rsv_pre_f', 'ar_date', 'ar_rsva_nd50']]
sera_filtered


,ptid,aliquot_id,rsv_a_outcome,rsv_b_outcome,ar_sars_msd_sc_date,ar_rsv_pre_f,ar_date,ar_rsva_nd50
303,20000401,3704633g,1,1,23332.0,126881.8,NaN,NaN
304,20000401,0027014g,1,1,NaN,NaN,23294.0,NaN
305,20000401,8189674g,1,1,22909.0,293.3,NaN,NaN
306,20000401,0027014g,1,1,NaN,NaN,23201.0,422.0
307,20000401,1665540g,1,1,NaN,NaN,23485.0,886.0
...,...,...,...,...,...,...,...,...
637,20074272,4210123g,0,1,23172.0,750.0,NaN,NaN
638,20074272,7047035g,0,1,NaN,NaN,23397.0,465.0
639,20074272,3883693g,0,1,NaN,NaN,23470.0,NaN
640,20074272,3883693g,0,1,NaN,NaN,23484.0,1474.0


### Convert the sas date format (days after Jan. 1, 1960) in ar_sars_msd_sc_date column to YYYY-MM-DD format

In [60]:
from datetime import datetime, timedelta

def sas_to_ymd(sas_date):
    if pd.isna(sas_date):
        return None
    sas_epoch = datetime(1960, 1, 1)
    converted_date = sas_epoch + timedelta(days=int(sas_date))
    return converted_date.strftime("%Y-%m-%d")

sera_filtered['collection_date_ymd'] = sera_filtered['ar_sars_msd_sc_date'].map(sas_to_ymd)
sera_filtered['neut_date_ymd'] = sera_filtered['ar_date'].map(sas_to_ymd)

sera_filtered.to_csv(sera_filtered_converted_date, index=False)

sera_df = sera_filtered






/tmp/ipykernel_58392/177214711.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sera_filtered['collection_date_ymd'] = sera_filtered['ar_sars_msd_sc_date'].map(sas_to_ymd)
/tmp/ipykernel_58392/177214711.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sera_filtered['neut_date_ymd'] = sera_filtered['ar_date'].map(sas_to_ymd)


# Filtering Swab Data

### Create a dataframe that contains only our selected ptids, aliquot ids, and dates

In [61]:

filtered_aliquots = selected_child[['ptid', 'aliquot_id', 'collect_dt']]
filtered_aliquots


,ptid,aliquot_id,collect_dt
0,20000401.0,3001262g,9/21/22
1,20000401.0,6125192g,11/18/23
2,20000533.0,8129615g,8/19/22
3,20000533.0,3614113g,8/21/23
4,20001363.0,2140292g,7/24/22
5,20001363.0,4446778g,9/10/23
6,20002793.0,9555599g,9/3/22
7,20002793.0,8242954g,9/23/23
8,20002903.0,4027265g,8/22/22
9,20002903.0,4636768g,9/2/23


### And then we can create a filtered dataset for when our ptids test positive for rsv a or b
I am filtering by the oa_rsv_a or oa_rsv_b column, but based on Teagan's plots, she seems to be filtering by something else. She has 3 positive events for RSV A for ptid 20001363, but they only have two if filtering the way that I am. I'll have to ask Cassie which column is actually correct. The 2022_46 week swab is the one that is missing by my filtering strategy. It's possible that I should just be filtering by the rsv_a and rsv_b columns? There is also a RSV_a_all column that combines the two I think...

In [62]:
swab_filtered = swab[swab['ptid'].isin(selected_ptids)]     #filter swab data to contain only our ptids
swab_positive = swab_filtered[(swab['oa_rsv_a'] == 1) | (swab['oa_rsv_b'] == 1)]        #filter for positive rsv a or b swabs
swab_positive[['ptid', 'swab_date', 'oa_rsv_a', 'oa_rsv_b']]

/tmp/ipykernel_58392/3104045497.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  swab_positive = swab_filtered[(swab['oa_rsv_a'] == 1) | (swab['oa_rsv_b'] == 1)]        #filter for positive rsv a or b swabs


,ptid,swab_date,oa_rsv_a,oa_rsv_b
3008,20000401,27-Sep-22,1.0,0.0
3009,20000401,2-Oct-22,1.0,0.0
3010,20000401,10-Oct-22,1.0,0.0
3011,20000401,16-Oct-22,1.0,0.0
3067,20000401,14-Nov-23,0.0,1.0
3103,20000533,27-Nov-22,0.0,1.0
3280,20001363,7-Nov-22,1.0,0.0
3295,20001363,19-Feb-23,1.0,0.0
3477,20002793,28-Jan-23,1.0,0.0
3539,20002793,6-Jan-24,0.0,1.0


### Now, we can merge the aliquot/date dataframe and the filtered swab dataframe to have the swab dates next to the aliquot dates

In [63]:
swab_aliquot_concat = pd.concat([swab_positive, filtered_aliquots], ignore_index=True)
swab_aliquot_concat = swab_aliquot_concat[['ptid', 'aliquot_id', 'collect_dt', 'oa_rsv_a', 'oa_rsv_b', 'swab_date']].sort_values(['ptid'])
swab_df = swab_aliquot_concat

In [ ]:


# Clean column names
swab_df.columns = swab_df.columns.str.strip()
sera_df.columns = sera_df.columns.str.strip()

# -----------------------------
# Convert date columns safely
# -----------------------------
swab_df.loc[:, "swab_date"] = pd.to_datetime(swab_df["swab_date"], errors="coerce")
sera_df.loc[:, "collection_date_ymd"] = pd.to_datetime(sera_df["collection_date_ymd"], errors="coerce")
sera_df.loc[:, "neut_date_ymd"] = pd.to_datetime(sera_df["neut_date_ymd"], errors="coerce")

# -----------------------------
# Handle ptid as nullable integer
# -----------------------------
swab_df.loc[:, "ptid"] = pd.to_numeric(swab_df["ptid"], errors="coerce").astype("Int64")
sera_df.loc[:, "ptid"] = pd.to_numeric(sera_df["ptid"], errors="coerce").astype("Int64")

# -----------------------------
# Ensure aliquot_id column is recognized
# -----------------------------
swab_df["aliquot_id"] = swab_df.get("aliquot_id")
swab_df.loc[:, "aliquot_id"] = swab_df["aliquot_id"].replace("", pd.NA)

# -----------------------------
# Create event_type for swab_df
# -----------------------------
swab_df.loc[:, "event_type"] = None
swab_df.loc[swab_df["oa_rsv_a"] == 1, "event_type"] = "RSV A pos."
swab_df.loc[swab_df["oa_rsv_b"] == 1, "event_type"] = "RSV B pos."
swab_df.loc[swab_df["aliquot_id"].notna(), "event_type"] = "Aliquot"

# -----------------------------
# Shared ptid dropdown
# -----------------------------
all_ptids = sorted(set(swab_df["ptid"].dropna().unique()) |
                   set(sera_df["ptid"].dropna().unique()))
ptid_dropdown = alt.binding_select(options=all_ptids, name="Select ptid: ")
ptid_selection = alt.selection_point(fields=["ptid"], bind=ptid_dropdown, value=all_ptids[0])

# -----------------------------
# Compute shared date range
# -----------------------------
min_date = min(
    swab_df["swab_date"].min(),
    sera_df["collection_date_ymd"].min(),
    sera_df["neut_date_ymd"].min()
)
max_date = max(
    swab_df["swab_date"].max(),
    sera_df["collection_date_ymd"].max(),
    sera_df["neut_date_ymd"].max()
)

# -----------------------------
# Set chart width
# -----------------------------
chart_width = 850

# -----------------------------
# Binding Plot (Top)
# -----------------------------
binding_chart = (
    alt.Chart(sera_df.dropna(subset=["collection_date_ymd", "ar_rsv_pre_f"]))
    .mark_line(point=True)
    .encode(
        x=alt.X(
            "collection_date_ymd:T",
            scale=alt.Scale(domain=[min_date, max_date]),
            axis=alt.Axis(format="%Y", tickCount="year", labelAngle=-45, title="Year")
        ),
        y=alt.Y("ar_rsv_pre_f:Q", title="Binding (ar_rsv_pre_f)"),
        tooltip=["ptid", "collection_date_ymd", "ar_rsv_pre_f"]
    )
    .transform_filter(ptid_selection)
    .properties(height=220, width=chart_width)
)

# -----------------------------
# Neutralization Plot (Middle)
# -----------------------------
neut_chart = (
    alt.Chart(sera_df.dropna(subset=["neut_date_ymd", "ar_rsva_nd50"]))
    .mark_line(point=True, color="green")
    .encode(
        x=alt.X(
            "neut_date_ymd:T",
            scale=alt.Scale(domain=[min_date, max_date]),
            axis=alt.Axis(format="%Y", tickCount="year", labelAngle=-45, title="Year")
        ),
        y=alt.Y("ar_rsva_nd50:Q", title="Neutralization (ar_rsva_nd50)"),
        tooltip=["ptid", "neut_date_ymd", "ar_rsva_nd50"]
    )
    .transform_filter(ptid_selection)
    .properties(height=220, width=chart_width)
)

# -----------------------------
# Swab Timeline Plot (Bottom) - evenly spaced events
# -----------------------------
swab_df.loc[:, "swab_date_str"] = swab_df["swab_date"].dt.strftime("%Y-%m-%d")

timeline_chart = (
    alt.Chart(swab_df.dropna(subset=["swab_date_str", "event_type"]))
    .mark_point(size=120)
    .encode(
        x=alt.X(
            "swab_date_str:N",  # categorical for even spacing
            sort=alt.SortField("swab_date", order="ascending"),
            title="Event Date (Evenly Spaced)"
        ),
        y=alt.value(50),
        color=alt.Color(
            "event_type:N",
            scale=alt.Scale(
                domain=["RSV A pos.", "RSV B pos.", "Aliquot"],
                range=["red", "blue", "black"]
            )
        ),
        shape=alt.Shape("event_type:N"),
        tooltip=["ptid", "event_type", "swab_date"]
    )
    .transform_filter(ptid_selection)
    .properties(height=600, width=chart_width)
)

# -----------------------------
# Stack charts vertically and add ptid selection only once
# -----------------------------
final_chart = (
    alt.vconcat(
        binding_chart,
        neut_chart,
        timeline_chart
    )
    .add_params(ptid_selection)  # only here
    .resolve_scale(x="shared")
    .properties(title="Patient Timeline: Binding, Neutralization, and Swab Events")
)


#final_chart.save(timeline_stacked)
final_chart

/tmp/ipykernel_58392/785073526.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  swab_df.loc[:, "swab_date"] = pd.to_datetime(swab_df["swab_date"], errors="coerce")
/tmp/ipykernel_58392/785073526.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[20000401, 20000401, 20000401, 20000401, 20000401, 20000401, 20000401,
 20000533, 20000533, 20000533, 20001363, 20001363, 20001363, 20001363,
 20002793, 20002793, 20002793, 20002793, 20002903, 20002903, 20002903,
 20002903, 20003670, 20003670, 20003670, 20003670, 20003670, 20005534,
 20005534, 20005534, 20005534, 20005534, 20005534, 20005704, 20005704,
 20005704, 20005704, 20005704, 20005704, 20005704, 20005704, 20011434,
 20011434, 20011434, 20011434, 20011434, 20011434, 20011982, 20011982,
 20011982, 20011982, 2001

TypeError: '<=' not supported between instances of 'Timestamp' and 'float'

### Now, we can plot the data on a timeline

In [ ]:
df["swab_date"]  = pd.to_datetime(df["swab_date"], errors="coerce")
df["collect_dt"] = pd.to_datetime(df["collect_dt"], errors="coerce")

df["event_date"] = df["swab_date"].combine_first(df["collect_dt"])

df["event_type"] = None
df.loc[df["oa_rsv_a"] == 1, "event_type"] = "RSV A pos."
df.loc[df["oa_rsv_b"] == 1, "event_type"] = "RSV B pos."
df.loc[df["aliquot_id"].notna(), "event_type"] = "Aliquot"

timeline_df = df.dropna(subset=["event_type", "event_date"]).copy()
timeline_df["ptid"] = timeline_df["ptid"].astype(int)

# --- STEP 1: CREATE DATE LABEL STRING ---
# This goes *after* timeline_df is made, before charting.
timeline_df["date_label"] = timeline_df["event_date"].dt.strftime("%Y-%m-%d")


# --- STEP 2: Make Dropdown Selection ---
ptid_dropdown = alt.binding_select(
    options=sorted(timeline_df["ptid"].unique()),
    name="Select ptid: "
)

ptid_selection = alt.selection_point(
    fields=["ptid"],
    bind=ptid_dropdown,
    value=sorted(timeline_df["ptid"].unique())[0]
)



# --- STEP 3: BUILD ALTair CHART ---
chart = (
    alt.Chart(timeline_df)
    .mark_point(size=120)
    .encode(
        x=alt.X(
            "date_label:N",
            title="Date (YYYY-MM-DD)",
            sort=alt.SortField(field="event_date", order="ascending"),
            axis=alt.Axis(
                labelAngle=-45,
                labelFontSize=14,
                titleFontSize=18
            ),
        ),
        y=alt.value(50),
        color=alt.Color(
            "event_type:N",
            scale=alt.Scale(
                domain=["RSV A pos.", "RSV B pos.", "Aliquot"],
                range=["red", "blue", "black"]
            ),
            legend=alt.Legend(
                title="Event Type",
                labelFontSize=14,
                titleFontSize=18
                )
        ),
        shape=alt.Shape(
            "event_type:N",
            scale=alt.Scale(
                domain=["RSV A pos.", "RSV B pos.", "Aliquot"],
                range=["circle", "square", "triangle"]
            )
        ),
        tooltip=["ptid", "event_type", "event_date"]
    )
    .add_params(ptid_selection)
    .transform_filter(ptid_selection)
    .properties(
        width=800, height=100,
        title=alt.TitleParams(
            text="Timeline of Positive Swab Events and Selected Aliquots",
            fontSize=20,
            anchor='middle'
        )
        )
)

chart.save(aliquot_swab_timeline)
chart


alt.Chart(...)